In this notebook the selected point between consecutive -/+ or (+/- or  -/+) is the one with the lowest abs of the first derivate (for waves) or second derivate (for peaks and valleys)

THERE IS A PROBLEM WITH THE DERIVATES, THEY ARE NOT EAQUAL WITH MATHEMATICA. MUST BE LIKE IN MATHEMATICA

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
from scipy.ndimage.filters import gaussian_filter

/var/folders/_3/sryc3_nj66vgwtq9swf_681m0000gn/T/ipykernel_6266/2300134294.py:5: DeprecationWarning: Please use `gaussian_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter


In [2]:
#reading the data
df=pd.read_csv("/Users/linaruiz/Documents/projectEpidemicCurve/data/uncover_countries.csv")
df.head()

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
0,2020-01-03,BE,Belgium,EURO,0,0,0,0
1,2020-01-04,BE,Belgium,EURO,0,0,0,0
2,2020-01-05,BE,Belgium,EURO,0,0,0,0
3,2020-01-06,BE,Belgium,EURO,0,0,0,0
4,2020-01-07,BE,Belgium,EURO,0,0,0,0


In [3]:
#selecting italy country
df_italy = df[df['Country_code'] == 'IT'].reset_index(drop=True)
df_italy['Date_reported'] = pd.to_datetime(df_italy['Date_reported'])
df_italy.head()

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
0,2020-01-03,IT,Italy,EURO,0,0,0,0
1,2020-01-04,IT,Italy,EURO,0,0,0,0
2,2020-01-05,IT,Italy,EURO,0,0,0,0
3,2020-01-06,IT,Italy,EURO,0,0,0,0
4,2020-01-07,IT,Italy,EURO,0,0,0,0


In [5]:
df_max_scaled = df_italy.copy()
# apply normalization techniques on Column 1
df_max_scaled["norm_new_cases"] = df_max_scaled["New_cases"]/df_max_scaled["New_cases"].abs().max()
df_max_scaled["norm_new_cases"].to_csv("normalizadosPython.csv",index=False)
df_max_scaled["norm_new_cases"]

0      0.000000
1      0.000000
2      0.000000
3      0.000000
4      0.000000
         ...   
994    0.098732
995    0.092419
996    0.098017
997    0.082385
998    0.043862
Name: norm_new_cases, Length: 999, dtype: float64

In [6]:
#applying gaussian filter to new_cases
b=28/2
df_max_scaled["smoothed_new_cases_2"] = gaussian_filter(df_max_scaled['norm_new_cases'], b)
df_max_scaled["smoothed_new_cases_2"].to_csv("suavizadosPyhton.csv",index=False)
df_max_scaled["smoothed_new_cases_2"]

0      1.966053e-07
1      2.140056e-07
2      2.425934e-07
3      2.979717e-07
4      3.652036e-07
           ...     
994    7.797321e-02
995    7.791185e-02
996    7.786803e-02
997    7.783993e-02
998    7.782736e-02
Name: smoothed_new_cases_2, Length: 999, dtype: float64

In [ ]:
#plotting
plt.figure(figsize=(12,12))
plt.plot(df_max_scaled["Date_reported"],df_max_scaled["norm_new_cases"],color="gray")

plt.plot(df_max_scaled["Date_reported"],df_max_scaled["smoothed_new_cases_2"],color="red")
plt.ylabel('numbers')
plt.xticks(rotation=60)
plt.show()

In [ ]:
#df_max_scaled["norm_new_cases"]
#print(df_max_scaled["norm_new_cases"])
#print(len(df_max_scaled["norm_new_cases"]))
#print(len(df_max_scaled["norm_new_cases"][1:]))
#print(df_max_scaled["norm_new_cases"][1:])
#print(len(df_max_scaled["norm_new_cases"][:-1]))
#print(df_max_scaled["norm_new_cases"][:-1])
#print(df_max_scaled["norm_new_cases"][998]-df_max_scaled["norm_new_cases"][997])


In [ ]:
#getting the first derivate discrete of the smoothed new cases
df_max_scaled["firstDerivate_new_cases"] = df_max_scaled["smoothed_new_cases_2"].rolling(2).agg(lambda x : x.iloc[1]-x.iloc[0])
print(df_max_scaled["firstDerivate_new_cases"])

In [ ]:
#applying gaussian filter to "the first derivate discrete of the smoothed new cases"
b=15/2
df_max_scaled["smoothedFD_new_cases"] = gaussian_filter(df_max_scaled['firstDerivate_new_cases'], b)

In [ ]:
#plotting
plt.figure(figsize=(12,12))

plt.plot(df_max_scaled["Date_reported"],df_max_scaled["smoothedFD_new_cases"] ,color="red")
plt.plot(df_max_scaled["Date_reported"],df_max_scaled["firstDerivate_new_cases"],color="Black")


plt.ylabel('numbers')
plt.xticks(rotation=60)
plt.show()

In [ ]:
#getting the second derivate: discreted derivate of the smoothed first derivate
df_max_scaled["secondDerivate_new_cases"] = df_max_scaled["smoothedFD_new_cases"].rolling(2).agg(lambda x : x.iloc[1]-x.iloc[0])

In [ ]:
df_max_scaled[["firstDerivate_new_cases","smoothedFD_new_cases","secondDerivate_new_cases"]].to_csv("derivatesPython.csv",index=False)

In [ ]:
#plotting
plt.figure(figsize=(12,12))

plt.plot(df_max_scaled["Date_reported"],df_max_scaled["secondDerivate_new_cases"],color="Black")


plt.ylabel('numbers')
plt.xticks(rotation=60)
plt.show()

To detect waves we used the first derivated smooted and second derivate: 

    1. detect consecutive NEGATIVE-POSITIVE

In [ ]:
df_max_scaled["smoothedFDrolling"]= (df_max_scaled["smoothedFD_new_cases"].rolling(2).agg(lambda x : True if x.iloc[0]<0 and x.iloc[1]>0 else False)).fillna(False)
df_max_scaled["smoothedFDrolling"]

In [ ]:
#(df_max_scaled["smoothedFDrolling"]==False).sum()
#(df_max_scaled["smoothedFDrolling"]==True).sum()
#(df_max_scaled["smoothedFDrolling"].isna()).sum()
#df_max_scaled["smoothedFDrolling"].value_counts()

In [ ]:
# Alexis' aproximation
df_2=df_max_scaled[df_max_scaled["smoothedFDrolling"]==True][["Date_reported","smoothedFD_new_cases"]].reset_index(drop=True)
df_2

In [ ]:
df_3=df_max_scaled[df_max_scaled["Date_reported"].isin(list(df_2["Date_reported"] - datetime.timedelta(days=1)))][["Date_reported","smoothedFD_new_cases"]].reset_index(drop=True)
df_3.rename(columns={'Date_reported':'Date_reported_1',"smoothedFD_new_cases":"smoothedFD_new_cases_1"},inplace=True)
df_3

In [ ]:
#these are the consecutive NEGATIVE-POSITIVE
df_concatenate1=pd.concat([df_2, df_3],axis=1)
df_concatenate1

    2. select the date associated to each position (negative, positive) with the lowest Abs[smoothedFD]

In [ ]:
#the dates with the lowest Abs[smoothedFD_new_cases] associated
a1=list(df_concatenate1.agg(lambda x : x.Date_reported if abs(x.smoothedFD_new_cases)<abs(x.smoothedFD_new_cases_1)  else x.Date_reported_1,axis=1))
a1

In [ ]:
cutDays1=df_max_scaled[df_max_scaled["Date_reported"].isin(a1)].reset_index(drop = True)
cutDays1

In [ ]:
#Lina's approximation
#def selecting(a,b):
#     return df_max_scaled['Date_reported2'][a] if df_max_scaled["smoothedFD_new_cases"][a] < df_max_scaled["smoothedFD_new_cases"][b] else df_max_scaled['Date_reported2'][b]
    
#%time
#positions1 = np.where(df_max_scaled["smoothedFDrolling"]==True)[0] #positive
#positions2= positions1-1 #negative
#cutDays=[selecting(a, b) for a, b in zip(positions1, positions2)]
#cutDays

    3. From this cutdays(dates) select those with the second derivate (discrete derivate of the first derivate )higher than the threshold 

In [ ]:
#this is set up by the user
thresholdP= 2.5*10**-5

In [ ]:
d0=cutDays1.agg(lambda x : x["Date_reported"] if  x["secondDerivate_new_cases"] > thresholdP else [],axis=1)
d0

In [ ]:
dates = pd.to_datetime(d0.explode())
dates=dates[~np.isnat(dates)]
dates

    4. Plot the days

In [ ]:
#fig, ax = plt.subplots()

cases =df_max_scaled["norm_new_cases"]

plt.figure(figsize=(12,12))


plt.plot(df_max_scaled["Date_reported"],cases,color="gray")
plt.plot(df_max_scaled["Date_reported"],df_max_scaled["smoothed_new_cases_2"],color="red")


#plt.ylabel('numbers')
#plt.xticks(rotation=60)
#plt.show()



# Plot vertical lines at each date
for date in dates:
    plt.axvline(x=date, color='black', ymax= max(cases), linestyle='--', linewidth=.91)

# Set the y-axis limits
#ax.set_ylim(0, 1)

# Hide the y-axis ticks and labels
#ax.set_yticks([])

# Show the plot
#plt.show()

To detect peaks/valleys we used the second derivate (discrete derivate of the first derivate ): 

    1.Detect POSITIVE-NEGATIVE and NEGATIVE-POSITIVE points in the second derivate:

In [ ]:
df_max_scaled["smoothedFDrolling2"]= (df_max_scaled["secondDerivate_new_cases"].rolling(2).agg(lambda x : True if (x.iloc[0]<0 and x.iloc[1]>0) or (x.iloc[0]>0 and x.iloc[1]<0) else False)).fillna(False)
df_max_scaled["smoothedFDrolling2"]

In [ ]:
print((df_max_scaled["smoothedFDrolling2"]==False).sum())
print((df_max_scaled["smoothedFDrolling2"]==True).sum())
print((df_max_scaled["smoothedFDrolling2"].isna()).sum())
print(df_max_scaled["smoothedFDrolling2"].value_counts())

In [ ]:
df_4=df_max_scaled[df_max_scaled["smoothedFDrolling2"]==1][["Date_reported","secondDerivate_new_cases"]].reset_index(drop=True)
df_4

In [ ]:
df_5=df_max_scaled[df_max_scaled["Date_reported"].isin(list(df_4["Date_reported"] - datetime.timedelta(days=1)))][["Date_reported","secondDerivate_new_cases"]].reset_index(drop=True)
df_5.rename(columns={'Date_reported':'Date_reported_1',"secondDerivate_new_cases":"secondDerivate_new_cases_1"},inplace=True)
df_5

In [ ]:
#these are the consecutive +/- and -/+
df_concatenate2=pd.concat([df_4, df_5], axis=1)
df_concatenate2

    2. Select from each consequitve -/+ or +/- the date with the lowest abs[second derivate]

In [ ]:
a2= list(df_concatenate2.agg(lambda x : x.Date_reported if abs(x.secondDerivate_new_cases)<abs(x.secondDerivate_new_cases_1)  else x.Date_reported_1,axis=1))
a2

In [ ]:
cutDays2=(df_max_scaled[df_max_scaled["Date_reported"].isin(a2)].reset_index(drop = True))[["Date_reported","secondDerivate_new_cases"]]
cutDays2#this is the dataframe when the first number of the second derivate is positive

    3. Now we have only one date associated to each cut point. The set of cut points are pairs that delimited the peaks or valleys. Now we should select the consecutive pair with an associated lowest second derivate lower than the Negative Threshold:

        NOTE: Remember that we first take the lowest second derivate between the consecutive cutdays, and then this lowest value is compared with the negative threshold. In other words, we looked for the lowest second derivate values in the set of negative values between a pair of cutdays. The pairs are organized depending on the sign of the first value of the second derivate:

        a. for those with the positive sign: the consecutive pair start in the first cutdays2 set of dates

In [ ]:
signFirstValueSecondD="positive" if df_5["secondDerivate_new_cases_1"][0] > 0 else "negative"
signFirstValueSecondD# this is important to define the consequitive cut points that have only negative numbers

In [ ]:
thresholdN= -9*10**-5 #thr negative
thresholdN

In [ ]:
cutDays22 = cutDays2["Date_reported"]
cutDays22

In [ ]:
d=cutDays22.groupby(cutDays22.index//2).agg(lambda x : x if  min(
        df_max_scaled[
            (df_max_scaled["Date_reported"]>=x.iloc[0]) & 
            (df_max_scaled["Date_reported"]<=x.iloc[1])
        ]["secondDerivate_new_cases"]# in this way we get all the second derivate values between consecutive pair of cutdays
) < thresholdN else [])
d

In [ ]:
dates = pd.to_datetime(d.explode())
dates=dates[~np.isnat(dates)]
dates

In [ ]:
cases =df_max_scaled["norm_new_cases"]
plt.figure(figsize=(12,12))

plt.plot(df_max_scaled["Date_reported"],cases,color="gray")
plt.plot(df_max_scaled["Date_reported"],df_max_scaled["smoothed_new_cases_2"],color="red")

plt.ylabel('numbers')
plt.xticks(rotation=60)

# Plot vertical lines at each date
for date in dates :
    plt.axvline(x=date, color='black', ymax= max(cases), linestyle='--', linewidth=.91)

        b. for those with the negative sign:  the consecutive pair start and finalize in the first and last Date_reported, respectivelly

In [ ]:
cutDays23=pd.concat([df_max_scaled["Date_reported"].iloc[[0]],cutDays22,df_max_scaled["Date_reported"].iloc[[-1]]]).reset_index(drop=True)
cutDays23

In [ ]:
d2=cutDays23.groupby(cutDays23.index//2).agg(lambda x : x if  min(
        df_max_scaled[
            (df_max_scaled["Date_reported"]>=x.iloc[0]) & 
            (df_max_scaled["Date_reported"]<=x.iloc[1])
        ]["secondDerivate_new_cases"]) < thresholdN else [])
d2

In [ ]:
dates2 = pd.to_datetime(d2.explode())
dates2=dates2[~np.isnat(dates2)]
dates2

In [ ]:
plt.figure(figsize = (12,12))
plt.plot(df_max_scaled["Date_reported"], df_max_scaled["norm_new_cases"], color="gray")
plt.plot(df_max_scaled["Date_reported"],df_max_scaled["smoothed_new_cases_2"],color="red")
for date in dates2:
    plt.axvline(x=date, color="black", ymax=max(df_max_scaled["norm_new_cases"]), linestyle='--', linewidth=.91)

In [ ]:
#Radio
n0 = 1000
tm = 
function = lambda x: n0 * exp(-(ln(2)/tm))